In [40]:
import os
from groq import Groq
from dotenv import load_dotenv


In [34]:
from langchain_groq import ChatGroq

In [57]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key='GROQ_API_KEY'  # optional if GROQ_API_KEY is set in env
)


In [58]:
response = llm.invoke([{"role": "user", "content": "hi"}])
print(response.content)

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [49]:
from langchain.tools import tool

In [50]:
@tool
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """
    return a * b

In [51]:
multiply

StructuredTool(name='multiply', description='Multiply two integers.\n\nArgs:\n    a (int): The first integer.\n    b (int): The second integer.\n\nReturns:\n    int: The product of a and b.', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x000002698CA42C20>)

In [52]:
from langchain_core.tools import StructuredTool

In [53]:
from pydantic import BaseModel
class WeatherInput(BaseModel):
    city: str

In [54]:
def get_weather(city: str) -> str:
    """
    Get the weather for a given city.

    Args:
        city (str): The name of the city.

    Returns:
        str: A string describing the weather in the city.
    """
    return f"The weather in {city} is sunny."

In [55]:
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="get_weather",
    description="Fetches real-time weather data for a city",
    args_schema=WeatherInput,  
)

In [56]:
from typing import ClassVar, Type
from pydantic import BaseModel, Field
from langchain.tools import StructuredTool  # if you're using LangChain

# Input schema
class WeatherInput(BaseModel):
    city: str = Field(..., description="City name")
    units: str = Field("metric", description="metric or imperial")

# Tool
class GetWeatherTool(StructuredTool):
    name: ClassVar[str] = "get_weather"
    description: ClassVar[str] = "Fetches weather data for a city"
    args_schema: ClassVar[Type[BaseModel]] = WeatherInput

    def _run(self, city: str, units: str = "metric") -> str:
        return get_weather(city, units)
